# TF-IDF를 활용한 Passage Retrieval 실습

* KorQuAD dataset에서 passage를 retrieval하기

## Requirements

In [ ]:
!pip install datasets
!pip install transformers

## 데이터셋 준비

* KorQuAD train 데이터셋을 search corpus로 활용

In [ ]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

* corpus 정의
  * 문서들만 가져오기
  * 중복된 context들이 있기 때문에 `set()` 후에 `list()`를 취함

In [ ]:
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus) # 9606

## 토크나이저 준비

* 가장 기본적인 띄어쓰기를 기준으로 token을 나누는 tokenizer 정의
  * 성능 향상을 위해 더 세밀한 tokenizer 활용 가능함

In [ ]:
tokenizer_func = lambda x: x.split(' ')

In [ ]:
tokenizer_func(corpus[0])[:10]

## TF-IDF embedding 만들기

* Scikit-learn의 TfidfVectorizer library를 활용하여 TF-IDF embeddind 만들기 (unigram, bigram 활용)
  * text를 vector로 바꿔주는 class

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2))

* `fit()`
  * scikit-learn에서 일종의 학습 방법(neural network 학습과는 다름)
  * 문서 전체를 보고 IDF에 해당되는 값을 구하고 term을 정의(vocabulary을 만듬)

* `transform()`
  * corpus를 vector로 바꿔줌

In [ ]:
vectorizer.fit(corpus)
sp_matrix = vectorizer.transform(corpus)

In [ ]:
sp_matrix.shape # (9606, 1272768) # (문서의 수, 지문 내에 등장하는 문서를 전부 가져온 수)

* 첫 번째 문서에 해당되는 값을 table화해서 보여줌(visualize)

In [ ]:
import pandas as pd
df = pd.DataFrame(sp_matrix[0].T.todense(), index=vectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(10))

## TF-IDF embedding을 활용하여 passage retrieval 실습하기

* random하게 하나의 지문을 가져와서 passage retrieval 수행
  * 기존 vectorizer를 사용할 것이기 때문에 train dataset에서 가져와야함
  * dev에서 가져오는 경우 context가 겹치지 않아서 정답 지문을 찾을 수 없음

In [ ]:
import random
import numpy as np

random.seed(1)
sample_idx = random.choice(range(len(dataset['train'])))

query = dataset['train'][sample_idx]['question']
ground_truth = dataset['train'][sample_idx]['context']

* query를 TF-IDF vector로 변환함

In [ ]:
query_vec = vectorizer.transform([query])

In [ ]:
query_vec.shape # (1, 1272768) # 1 : 편의상 matrix로 표현하기 위해 표기함

* 변환된 query vecto를 document들의 vector과 dot product를 수행함 => Document들의 similarity ranking을 구함

In [ ]:
result = query_vec * sp_matrix.T
result.shape # (1, 9606) # 9606 : 각 지문과 현재 지문의 유사도를 9606개의 숫자로 나타냄

* 유사도에서 가장 높은 숫자 찾기

In [ ]:
sorted_result = np.argsort(-result.data) # 오름차순으로 sort하는 경우 '-'를 붙임
doc_scores = result.data[sorted_result] # sorted_result가 index이기 때문에 indexing을 하여 score를 가져옴
doc_ids = result.indices[sorted_result]

* Top 3 개 확인

In [ ]:
k = 3
doc_scores[:k], doc_ids[:k]

In [ ]:
print("[Search query]\n", query, "\n")

print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
  doc_id = doc_ids[i]
  print(corpus[doc_id], "\n")
